In [1]:
import time
import pandas as pd
import numpy as np

# Technical analysis libraries
import MetaTrader5 as mt5
import talib  # Technical Analysis library for indicators & patterns

# News sentiment libraries
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# === Configuration ===
SYMBOL = "XAUUSD"
TIMEFRAME = mt5.TIMEFRAME_M15   # 15-minute bars (configurable)
LOT_SIZE = 0.1                  # trade volume in lots (adjust based on risk)
SL_RATIO = 0.005               # Stop-loss as a fraction of price (e.g., 0.005 = 0.5%)
TP_RATIO = 0.010               # Take-profit as a fraction of price (e.g., 0.01 = 1.0%)

# Account credentials (replace with your MT5 account details)
MT5_LOGIN = 12345678        # your MT5 account number
MT5_PASSWORD = "your_password"
MT5_SERVER = "YourBroker-Server"  # broker's server name

# News API configuration (replace with your API key for news feed)
NEWS_API_KEY = "YOUR_NEWSAPI_KEY"
NEWS_QUERY = "gold OR XAUUSD"   # query keywords for news search
NEWS_LANGUAGE = "en"
NEWS_SOURCES = "bloomberg,reuters,financial-times"  # example news sources


In [2]:
!python --version


Python 3.12.7


In [3]:
import talib
print(talib.get_functions()[:5])  # should print some indicator names


['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE']


In [4]:
import time
import pandas as pd
import numpy as np

# Technical analysis libraries
import MetaTrader5 as mt5
import talib  # Technical Analysis library for indicators & patterns

# News sentiment libraries
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# === Configuration ===
SYMBOL = "XAUUSD"
TIMEFRAME = mt5.TIMEFRAME_M15   # 15-minute bars (configurable)
LOT_SIZE = 0.1                  # trade volume in lots (adjust based on risk)
SL_RATIO = 0.005               # Stop-loss as a fraction of price (e.g., 0.005 = 0.5%)
TP_RATIO = 0.010               # Take-profit as a fraction of price (e.g., 0.01 = 1.0%)

# Account credentials (replace with your MT5 account details)
MT5_LOGIN = 10006442031       # your MT5 account number
MT5_PASSWORD = "FhT!SwK7"
MT5_SERVER = "MetaQuotes-Demo"  # broker's server name

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests
import numpy as np
import time

# === News API configuration ===
NEWS_API_KEY = "e4bb6e7f5c4c4f0691a5fd20e1b8ae3c"  # Replace with your real key
NEWS_QUERY = "gold OR XAUUSD OR Federal Reserve OR inflation OR dollar OR interest rates"
NEWS_LANGUAGE = "en"
NEWS_SOURCES = "bloomberg,reuters,financial-times,cnn,cnbc"

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def fetch_news_sentiment():
    """
    Fetch recent news articles about gold/USD and compute a sentiment score.
    Returns a tuple: (sentiment_score, headlines_list).
    """
    try:
        url = (
            f"https://newsapi.org/v2/everything?q={NEWS_QUERY}"
            f"&language={NEWS_LANGUAGE}&sortBy=publishedAt"
            f"&apiKey={NEWS_API_KEY}&pageSize=5"
        )
        response = requests.get(url, timeout=5)
        data = response.json()
    except Exception as e:
        print(f"News API request failed: {e}")
        return 0, []

    headlines = []
    sentiment_scores = []

    if data.get("status") == "ok":
        for article in data.get("articles", [])[:5]:
            title = article.get("title") or ""
            description = article.get("description") or ""
            text = title + ". " + description
            if text.strip():
                score = analyzer.polarity_scores(text)
                sentiment_scores.append(score["compound"])
                headlines.append(title)

    avg_sentiment = np.mean(sentiment_scores) if sentiment_scores else 0
    return avg_sentiment, headlines

# === Cache system ===
last_news_fetch = 0
sentiment_cache = (0, [])  # default neutral sentiment

# === Use this block inside your main loop ===
if time.time() - last_news_fetch > 900:  # Every 15 minutes
    sentiment_cache = fetch_news_sentiment()
    last_news_fetch = time.time()

sentiment_score, headlines = sentiment_cache



In [5]:
# Initialize MT5 and login to trading account
if not mt5.initialize():
    raise RuntimeError(f"MT5 initialize() failed, error code: {mt5.last_error()}")

authorized = mt5.login(MT5_LOGIN, password=MT5_PASSWORD, server=MT5_SERVER)
if not authorized:
    raise RuntimeError(f"MT5 login failed, error code: {mt5.last_error()}")

# Ensure the symbol is available
symbol_info = mt5.symbol_info(SYMBOL)
if symbol_info is None:
    raise RuntimeError(f"Symbol {SYMBOL} not found, please check symbol name.")
if not symbol_info.visible:
    mt5.symbol_select(SYMBOL, True)


In [6]:
# Candlestick pattern detection using TA-Lib
def detect_engulfing(open_series, high_series, low_series, close_series):
    """Detects bullish or bearish engulfing pattern on the latest candle.
    Returns +1 for bullish engulfing, -1 for bearish engulfing, 0 if none."""
    result = talib.CDLENGULFING(open_series, high_series, low_series, close_series)
    signal = result[-1]  # last value for the latest candle
    if signal > 0:
        return 1   # bullish engulfing
    elif signal < 0:
        return -1  # bearish engulfing
    else:
        return 0   # no pattern

def detect_doji(open_series, high_series, low_series, close_series):
    """Detects a Doji on the latest candle. Returns True if a doji pattern is found."""
    result = talib.CDLDOJI(open_series, high_series, low_series, close_series)
    return bool(result[-1] != 0)  # TA-Lib returns non-zero if doji

def detect_morning_star(open_series, high_series, low_series, close_series):
    """Detects a Morning Star pattern. Returns True if pattern found."""
    result = talib.CDLMORNINGSTAR(open_series, high_series, low_series, close_series, penetration=0.3)
    return bool(result[-1] != 0)  # non-zero indicates the pattern

def detect_evening_star(open_series, high_series, low_series, close_series):
    """Detects an Evening Star pattern. Returns True if pattern found."""
    result = talib.CDLEVENINGSTAR(open_series, high_series, low_series, close_series, penetration=0.3)
    return bool(result[-1] != 0)


In [7]:
def calculate_indicators(df):
    """Calculate technical indicators (RSI, MACD, MA, Bollinger) on price DataFrame.
    Returns a dict of latest indicator values needed for decision-making."""
    close = df['close'].astype(float).values  # TA-Lib expects float array
    # RSI (14-period)
    rsi_series = talib.RSI(close, timeperiod=14)
    latest_rsi = rsi_series[-1]

    # MACD (12,26,9 are standard periods)
    macd_series, macd_signal_series, macd_hist_series = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    latest_macd = macd_series[-1]
    latest_macd_signal = macd_signal_series[-1]
    latest_macd_hist = macd_hist_series[-1]

    # Moving Averages (e.g., 50-period SMA and 200-period EMA for trend)
    sma50_series = talib.SMA(close, timeperiod=50)
    ema200_series = talib.EMA(close, timeperiod=200)
    latest_sma50 = sma50_series[-1]
    latest_ema200 = ema200_series[-1]

    # Bollinger Bands (20-period, 2 std dev)
    upper, middle, lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    latest_upper = upper[-1]
    latest_middle = middle[-1]
    latest_lower = lower[-1]

    return {
        'rsi': latest_rsi,
        'macd': latest_macd,
        'macd_signal': latest_macd_signal,
        'macd_hist': latest_macd_hist,
        'sma50': latest_sma50,
        'ema200': latest_ema200,
        'boll_upper': latest_upper,
        'boll_mid': latest_middle,
        'boll_lower': latest_lower
    }


In [8]:
# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def fetch_news_sentiment():
    """
    Fetch recent news articles about gold/USD and compute a sentiment score.
    Returns a tuple: (sentiment_score, headlines_list).
    """
    try:
        url = (
            f"https://newsapi.org/v2/everything?q={NEWS_QUERY}"
            f"&language={NEWS_LANGUAGE}&sortBy=publishedAt&apiKey={NEWS_API_KEY}&pageSize=5"
        )
        response = requests.get(url, timeout=5)
        data = response.json()
    except Exception as e:
        print(f"News API request failed: {e}")
        return 0, []  # default to neutral sentiment on failure

    headlines = []
    sentiment_scores = []
    if data.get("status") == "ok":
        for article in data.get("articles", [])[:5]:
            title = article.get("title") or ""
            description = article.get("description") or ""
            text = title + ". " + description
            if text.strip():
                # Analyze sentiment of the combined title+description
                score = analyzer.polarity_scores(text)
                sentiment_scores.append(score['compound'])
                headlines.append(title)
    # Compute average sentiment (compound score between -1 and 1)
    avg_sentiment = np.mean(sentiment_scores) if sentiment_scores else 0
    return avg_sentiment, headlines


In [ ]:
# Utility function to get recent market data
def get_price_data(symbol, timeframe, n=100):
    """Retrieve the last n candles for the given symbol and timeframe as a DataFrame."""
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None:
        raise RuntimeError(f"Failed to retrieve market data for {symbol}: {mt5.last_error()}")
    df = pd.DataFrame(rates)
    # Convert timestamp to datetime
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# Determine initial position state
have_position = False  # track if we currently hold an open position

print("Starting trading loop. Press Ctrl+C to stop the bot.")
try:
    while True:
        # 1. Data Retrieval
        df = get_price_data(SYMBOL, TIMEFRAME, n=200)
        # Calculate indicators
        indicators = calculate_indicators(df)
        # Get candlestick pattern signals
        open_ser = df['open']
        high_ser = df['high']
        low_ser = df['low']
        close_ser = df['close']

        engulf_signal = detect_engulfing(open_ser, high_ser, low_ser, close_ser)  # 1, -1, or 0
        doji = detect_doji(open_ser, high_ser, low_ser, close_ser)
        morning_star = detect_morning_star(open_ser, high_ser, low_ser, close_ser)
        evening_star = detect_evening_star(open_ser, high_ser, low_ser, close_ser)

        # 2. News Sentiment
        sentiment_score, news_headlines = fetch_news_sentiment()
        # Determine sentiment bias
        sentiment_bias = "neutral"
        if sentiment_score > 0.3:
            sentiment_bias = "bullish"
        elif sentiment_score < -0.3:
            sentiment_bias = "bearish"

        # Print debug info (you can disable in live use to reduce log noise)
        print(f"\nLatest close price: {close_ser.iloc[-1]:.2f}")
        print(f"Engulfing pattern signal: {engulf_signal}, Doji: {doji}, MorningStar: {morning_star}, EveningStar: {evening_star}")
        print(f"RSI: {indicators['rsi']:.2f}, MACD: {indicators['macd']:.2f}, MACD_signal: {indicators['macd_signal']:.2f}")
        print(f"MA50: {indicators['sma50']:.2f}, EMA200: {indicators['ema200']:.2f}")
        print(f"Bollinger Bands: lower={indicators['boll_lower']:.2f}, upper={indicators['boll_upper']:.2f}")
        print(f"News sentiment: {sentiment_score:.2f} ({sentiment_bias}), Headlines: {news_headlines[:1]}")  # print the latest headline for context

        # 3. Trading Decision Logic
        # Determine if we have a bullish or bearish trade signal
        trade_signal = None
        if engulf_signal == 1 or morning_star:  # bullish pattern detected
            # Additional confirmation: RSI oversold or MACD bullish or price near lower band
            if indicators['rsi'] < 40 or indicators['macd_hist'] > 0 or close_ser.iloc[-1] < indicators['boll_lower']:
                trade_signal = "buy"
        elif engulf_signal == -1 or evening_star:  # bearish pattern detected
            if indicators['rsi'] > 60 or indicators['macd_hist'] < 0 or close_ser.iloc[-1] > indicators['boll_upper']:
                trade_signal = "sell"
        # (We use 40/60 for RSI to be slightly less strict than 30/70, catching earlier; adjust as desired)

        # Check trend filter: avoid counter-trend trades if desired
        # For example, only allow buy if price > EMA200 (uptrend) or sell if price < EMA200 (downtrend)
        price = close_ser.iloc[-1]
        if trade_signal == "buy" and price < indicators['ema200']:
            # If price is below EMA200 (downtrend), we might skip a buy signal to avoid counter-trend
            trade_signal = None
        if trade_signal == "sell" and price > indicators['ema200']:
            # Skip sell in uptrend
            trade_signal = None

        # Check news sentiment filter:
        if trade_signal == "buy" and sentiment_bias == "bearish":
            # News strongly bearish against gold, avoid buying
            print("Buy signal overridden by bearish news sentiment.")
            trade_signal = None
        if trade_signal == "sell" and sentiment_bias == "bullish":
            print("Sell signal overridden by bullish news sentiment.")
            trade_signal = None

        # If a trade signal is present and we don't already have a position, execute trade
        if trade_signal and not have_position:
            order_type = mt5.ORDER_TYPE_BUY if trade_signal == "buy" else mt5.ORDER_TYPE_SELL
            direction = 1 if trade_signal == "buy" else -1

            # Compute SL and TP prices
            symbol_info = mt5.symbol_info(SYMBOL)
            point = symbol_info.point
            price = mt5.symbol_info_tick(SYMBOL).ask if trade_signal == "buy" else mt5.symbol_info_tick(SYMBOL).bid
            sl_price = price - direction * (price * SL_RATIO)  # subtract if buy, add if sell
            tp_price = price + direction * (price * TP_RATIO)

            # Prepare order request structure
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": SYMBOL,
                "volume": LOT_SIZE,
                "type": order_type,
                "price": price,
                "sl": sl_price,
                "tp": tp_price,
                "deviation": 20,
                "magic": 10001,  # an arbitrary ID for this bot
                "comment": "XAUUSD_bot",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_RETURN,
            }
            result = mt5.order_send(request)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"{trade_signal.upper()} order placed at price {price:.2f}, SL={sl_price:.2f}, TP={tp_price:.2f}")
                have_position = True
            else:
                print(f"Order send failed, retcode={result.retcode}")
                # If order failed, we do not set have_position, and we might log the error details for debugging
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"Order failed: {result._asdict()}")

        # If we already have a position, we could check for exit conditions or trailing stop adjustments here.
        # For simplicity, this bot relies on the preset SL/TP to exit. We assume one trade at a time.
        if have_position:
            # Check if position is closed (hit SL/TP or manually closed)
            pos_info = mt5.positions_get(symbol=SYMBOL)
            if pos_info is None or len(pos_info) == 0:
                # Position closed
                have_position = False
                print("Position closed (SL/TP hit or manually closed).")

        # 4. Wait/sleep before next iteration
        time.sleep(60)  # wait 1 minute before checking again (adjust as needed)
except KeyboardInterrupt:
    print("Bot stopped by user.")
finally:
    mt5.shutdown()


Starting trading loop. Press Ctrl+C to stop the bot.

Latest close price: 3311.38
Engulfing pattern signal: 0, Doji: True, MorningStar: False, EveningStar: False
RSI: 52.84, MACD: 1.89, MACD_signal: 2.26
MA50: 3304.81, EMA200: 3256.69
Bollinger Bands: lower=3303.92, upper=3320.14
News sentiment: 0.00 (neutral), Headlines: []

Latest close price: 3311.51
Engulfing pattern signal: 0, Doji: False, MorningStar: False, EveningStar: False
RSI: 52.99, MACD: 1.90, MACD_signal: 2.26
MA50: 3304.81, EMA200: 3256.69
Bollinger Bands: lower=3303.93, upper=3320.15
News sentiment: 0.00 (neutral), Headlines: []

Latest close price: 3312.45
Engulfing pattern signal: 0, Doji: False, MorningStar: False, EveningStar: False
RSI: 54.06, MACD: 1.87, MACD_signal: 2.18
MA50: 3305.20, EMA200: 3257.11
Bollinger Bands: lower=3303.91, upper=3319.99
News sentiment: 0.00 (neutral), Headlines: []

Latest close price: 3312.32
Engulfing pattern signal: 0, Doji: False, MorningStar: False, EveningStar: False
RSI: 53.91, M

In [12]:
# === CANDLESTICK PATTERN DETECTION (FIXED) ===
import numpy as np
import talib

def detect_engulfing(open_series, high_series, low_series, close_series):
    result = talib.CDLENGULFING(open_series, high_series, low_series, close_series)
    signal = np.array(result)[-1]  # convert to NumPy array to use negative indexing
    if signal > 0:
        return 1   # bullish engulfing
    elif signal < 0:
        return -1  # bearish engulfing
    else:
        return 0   # no pattern

def detect_doji(open_series, high_series, low_series, close_series):
    result = talib.CDLDOJI(open_series, high_series, low_series, close_series)
    return bool(np.array(result)[-1] != 0)  # fixed indexing

def detect_morning_star(open_series, high_series, low_series, close_series):
    result = talib.CDLMORNINGSTAR(open_series, high_series, low_series, close_series, penetration=0.3)
    return bool(np.array(result)[-1] != 0)  # fixed indexing

def detect_evening_star(open_series, high_series, low_series, close_series):
    result = talib.CDLEVENINGSTAR(open_series, high_series, low_series, close_series, penetration=0.3)
    return bool(np.array(result)[-1] != 0)  # fixed indexing

In [13]:
if not mt5.initialize():
    print("MT5 disconnected. Attempting to reconnect...")
    if not mt5.initialize():
        raise RuntimeError(f"Reconnection failed: {mt5.last_error()}")
